<a href="https://colab.research.google.com/github/citruscaesar/building-footprint-extraction/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download Dependencies

In [ ]:
%%capture
!pip install wandb
!pip install lightning
!pip install torchmetrics
!pip install segmentation-models-pytorch
!pip install rasterio

In [ ]:
!git clone https://github.com/citruscaesar/building-footprint-extraction.git
!mkdir -p /content/src
!cp -r /content/building-footprint-extraction/src/ /content/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p ./inria/train

In [ ]:
!ls

In [ ]:
%%capture
'''
#!cp -r /content/drive/MyDrive/datasets/inria/AerialImageDataset/train /content/inria/train
#!cp -r /content/drive/MyDrive/datasets/inria/AerialImageDataset/tiled/* /content/inria/tiled
#!mkdir -p inria/tiled
'''

!mkdir -p ./inria/train
!cp /content/drive/MyDrive/datasets/inria/archive/tiled.zip ./inria  
!unzip inria/tiled.zip -d ./inria/

In [ ]:
!ls ./inria/

In [ ]:
from pathlib import Path
tiled = {
    "drive": Path("/content/drive/MyDrive/datasets/inria/AerialImageDataset/tiled/512x512"),
    "runtime": Path("/content/inria/tiled/512x512")
}
train_scenes = tiled["runtime"] / "train" / "scenes"

In [ ]:
assert (len(list(train_scenes.iterdir()))) == 18600

### Imports and Config

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pytorch_lightning as pl
import torchmetrics 
from torchmetrics import MetricCollection
from torchmetrics.classification import BinaryJaccardIndex, BinaryConfusionMatrix, BinaryF1Score, BinaryCohenKappa

import wandb
from pytorch_lightning.loggers import WandbLogger, CSVLogger

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from src.datamodule import InriaDataModule
from src.lightningmodule import SegmentationModule

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Pytorch Version: {torch.__version__}")
print(f"Lightning Version: {pl.__version__}")
print(f"Pytorch Running on : {DEVICE}")
#print(torch.__config__.show())

Pytorch Version: 1.13.1
Lightning Version: 2.0.0
Pytorch Running on : cuda


In [3]:
DATA = Path.cwd().parent / "datasets" / "inria" / "train" 
#DATA = Path("/content/drive/MyDrive/datasets/inria/AerialImageDataset/train")
#DATA = Path("/media/sambhav/30AC4696AC46568E/datasets/inaria/train")

In [4]:
print(list(DATA.iterdir()))
print(list((DATA.parent / "tiled").iterdir()))

[WindowsPath('c:/Users/Sambhav Chandra/dev/datasets/inria/train/gt'), WindowsPath('c:/Users/Sambhav Chandra/dev/datasets/inria/train/images')]
[WindowsPath('c:/Users/Sambhav Chandra/dev/datasets/inria/tiled/512x512')]


### Experiments

In [5]:
#TODO: Create Config Object, Setup to Return Config Dict 
#config dict
host = "local"
assert host in ["local", "colab"]
config = {
    "colab": {
        "tile_shape": (512, 512),
        "batch_size": 32,
        "accumulate": 0,
        "num_workers": 2,
        "learning_rate": 3e-4,
        "num_epochs": 5,
        "seed": 69
    },

    "local": {
        "tile_shape": (512, 512),
        "batch_size": 4,
        "accumulate": 4,
        "num_workers": 0,
        "learning_rate": 3e-4,
        "num_epochs": 1,
        "seed": 69
    }
}
config = config[host]

In [6]:
GLOBAL_SEED = pl.seed_everything(seed = config["seed"], workers = True)

Global seed set to 69


In [7]:
unet = smp.Unet("resnet18", encoder_weights="imagenet", in_channels=3, classes = 1)
preprocess = get_preprocessing_fn("resnet18", "imagenet")

In [8]:
model = SegmentationModule(unet, 
                           loss = "dice", 
                           learning_rate = config["learning_rate"], 
                           batch_size = config["batch_size"])

In [9]:
dm = InriaDataModule(DATA, 
                      preprocess_fn = preprocess,
                      tile_shape = config["tile_shape"], 
                      batch_size = config["batch_size"], 
                      num_workers = config["num_workers"])

In [10]:
'''
#os.environ["WANDB_NOTEBOOK_NAME"] = "experiments"
wandb.finish()
logs_dir = Path.cwd() / "logs"
logs_dir.mkdir(exist_ok = True)
wandblogger = WandbLogger(
    project = "building-segmentation-tests",
    save_dir = logs_dir 
) 
'''

'\n#os.environ["WANDB_NOTEBOOK_NAME"] = "experiments"\nwandb.finish()\nlogs_dir = Path.cwd() / "logs"\nlogs_dir.mkdir(exist_ok = True)\nwandblogger = WandbLogger(\n    project = "building-segmentation-tests",\n    save_dir = logs_dir \n) \n'

In [11]:
dev_trainer = pl.Trainer(
    accelerator = "auto",
    fast_dev_run = 1,
)

trainer = pl.Trainer(
    accelerator = "auto",
    #accumulate_grad_batches = config["accumulate"],
    #limit_train_batches = .25,
    max_epochs = config["num_epochs"],
    precision = 16,
    #logger = [wandblogger],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
c:\Users\Sambhav Chandra\.conda\envs\ml\lib\site-packages\lightning_fabric\connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
dev_trainer.fit(model, datamodule = dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | model        | Unet             | 14.3 M
1 | loss         | DiceLoss         | 0     
2 | val_metrics  | MetricCollection | 0     
3 | test_metrics | MetricCollection | 0     
4 | pred_metrics | MetricCollection | 0     
--------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.313    Total estimated model params size (MB)
c:\Users\Sambhav Chandra\.conda\envs\ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Sambhav Chandra\.conda\envs\ml\lib\site

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [ ]:
trainer.fit(model, datamodule = dm)

In [ ]:
checkpoint = torch.load("./checkpoints/segmodel-checkpoint.ckpt")
model.load_state_dict(checkpoint["state_dict"])

In [ ]:
dm_scene = InriaDataModule(DATA,
                           preprocess_fn=preprocess,
                           tile_shape=(512,512),
                           batch_size=1,
                           num_workers=config["num_workers"],
                           scene_transforms=transforms.Resize(1024),
                           mask_transforms=transforms.Resize(1024))

In [ ]:
dm_scene.prepare_data()
dm_scene.setup("predict")
ds = dm_scene.pred_dataset
dl = DataLoader(ds, batch_size=1, num_workers=0)
s, m = next(iter(dl))
print(len(dl))
print(s.shape)
print(m.shape)

25
torch.Size([1, 3, 1024, 1024])
torch.Size([1, 1, 1024, 1024])


In [13]:
trainer.test(model, datamodule = dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Sambhav Chandra\.conda\envs\ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    Test BinaryAccuracy    │     0.808814287185669     │
│    Test BinaryF1Score     │     0.092566579580307     │
│  Test BinaryJaccardIndex  │    0.04852939397096634    │
│         test/loss         │    0.7408815622329712     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.7408815622329712,
  'Test BinaryF1Score': 0.092566579580307,
  'Test BinaryJaccardIndex': 0.04852939397096634,
  'Test BinaryAccuracy': 0.808814287185669}]

In [17]:
trainer.test(model, dataloaders = dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Sambhav Chandra\.conda\envs\ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

c:\Users\Sambhav Chandra\.conda\envs\ml\lib\site-packages\pytorch_lightning\trainer\call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [19]:
predictions = trainer.predict(model, datamodule=dm_scene)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [23]:
predictions[0]

torch.Size([1, 1, 1024, 1024])

In [ ]:
trainer.save_checkpoint("./segmodel-checkpoint.ckpt")

In [ ]:
!cp ./segmodel-checkpoint.ckpt /content/drive/MyDrive/datasets/inria/

In [ ]:
wandb.finish()

In [ ]:
drive.flush_and_unmount()